In [123]:
# Matthew Berning
# import dependancies
import pandas as pd
from bs4 import BeautifulSoup # as bs
import requests
from requests import get
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import re
import string
import urllib

In [124]:
# URL of page to be scraped
mars_news_url = 'https://mars.nasa.gov/news/'
#  Retrieve page with the requests module
response = requests.get(mars_news_url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')
print(soup)

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxSC1oSUlkWbBMHXwRAHhdcCUA=","queueTime":0,"applicationTime":143,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQcPUlZTDxAFXVRUBQEPVA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]

In [125]:
#get all article titles (that are able to be scraped)
article_titles = soup.find_all('div', class_='content_title')
print(article_titles)

[<div class="content_title">
<a href="/news/8402/nasas-insight-places-first-instrument-on-mars/">
NASA's InSight Places First Instrument on Mars
</a>
</div>, <div class="content_title">
<a href="/news/8387/nasa-announces-landing-site-for-mars-2020-rover/">
NASA Announces Landing Site for Mars 2020 Rover
</a>
</div>, <div class="content_title">
<a href="/news/8348/opportunity-hunkers-down-during-dust-storm/">
Opportunity Hunkers Down During Dust Storm
</a>
</div>, <div class="content_title">
<a href="/news/8347/nasa-finds-ancient-organic-material-mysterious-methane-on-mars/">
NASA Finds Ancient Organic Material, Mysterious Methane on Mars
</a>
</div>, <div class="content_title">
<a href="/news/8326/nasa-invests-in-visionary-technology/">
NASA Invests in Visionary Technology 
</a>
</div>, <div class="content_title">
<a href="/news/8325/nasa-is-ready-to-study-the-heart-of-mars/">
NASA is Ready to Study the Heart of Mars
</a>
</div>]


In [126]:
#retrieve just the first article's title 
#(there is a problem with the BS codebase, it is somehow only scraping the infromation from the scrolling carosel on the very bottom of the page)

first_article = soup.find('div', class_='content_title').get_text()
print(first_article)



NASA's InSight Places First Instrument on Mars




In [127]:
#get article description
first_article_description = soup.find('div', class_='rollover_description_inner').get_text()
print(first_article_description)


In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.



In [128]:
#url for featured image to be scraped
space_images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# create a new chrome session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(space_images_url)

#use selenium to click button 'full image'
python_button = driver.find_element_by_id('full_image') 
python_button.click() #click  link

#once 'full image' is opened click 'more info' button to get to ACTUAL page with the full res image
python_button = driver.find_element_by_partial_link_text('more info')
python_button.click() #click link

In [129]:
#on the new page use beautifulsoup to get page source
soup_level1=BeautifulSoup(driver.page_source, 'lxml')

#test the soup to find full resloution image
# soup_level1.body.find_all('img', class_ ='main_image')

#pull out image 'src' by class and 'img' tag
img_find_src = [i['src'] for i in soup_level1.findAll('img', {'class': 'main_image'})]
# from STACKOVERFLOW: https://stackoverflow.com/questions/8289957/python-2-7-beautiful-soup-img-src-extract 

#cast soup.find to string
img_str = str(img_find_src)
#need to remove brackets and appostrophies from string generated by above
whitelist = string.ascii_letters + string.digits + '/' + '_' + '.'
img_url_end = ''
for char in img_str:
    if char in whitelist:
        img_url_end += char
    else:
        img_url_end += ''
#from STACKOVERFLOW: https://stackoverflow.com/questions/875968/how-to-remove-symbols-from-a-string-with-python        
       
img_url_start = 'https://www.jpl.nasa.gov' #begining of url
featured_img_url = (img_url_start + img_url_end) #FEATURED IMAGE FULL RESOLOUTION URL
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18452_hires.jpg


In [130]:
#the 'featured image' from the home work link is not always mars, even though it should be, 
#so it will just be a featured space image that will be put at the bottom of the finished webpage along with the other mars stuff

#capture image title
featured_image_title = soup_level1.find('h1', class_="article_title").text
#test
# print(featured_image_title)

#capture using beautiful soup all paragraph elements 
paragraphs = soup_level1.find_all('p')

#create empty list to contain p elements
paragraph_list = []

#iterate through retrieved paragraphs and conditinally append to paragraph list
for paragraph in paragraphs:
    p = [] #make empty list to contain text from each paragraph element individually
    p.append(paragraph.text)
    if any("Mission:" in s for s in p): #from STACKOVERFLOW: https://stackoverflow.com/questions/4843158/check-if-a-python-list-item-contains-a-string-inside-another-string
        break #end the loop when infromation from the 'image details' section of p elements is entered
    else:
        paragraph_list.append(paragraph) #append each paragraph to the paragraph list
#test
print(featured_image_title)
for i in range(len(paragraph_list)):
    print(paragraph_list[i].text)
    print(str(i) + "------------")


				A Puzzling Cosmic Ring			  
The Herschel Space Observatory has uncovered a weird ring of dusty material while obtaining one of the sharpest scans to date of a huge cloud of gas and dust, called NGC 7538. The gigantic ring structure is situated at the center-top of this image. The odd ovoid possesses the mass of 500 suns, with its long axis spanning about 35 light-years and its short axis about 25 light-years. 
0------------
Astronomers often see ring and bubble-like structures in cosmic dust clouds. The strong winds cast out by the most massive stars, called O-type stars, can generate these expanding puffs, as can their explosive deaths as supernovas. But no energetic source or remnant of a deceased O-type star, such as a neutron star, is apparent within the center of the ring. It is possible that a big star blew the bubble and, because stars are all in motion, subsequently left the scene, escaping detection. 
1------------
Astronomers study stellar nurseries such as NGC 7538 to b

In [136]:
#get mars weather from twitter
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'

response = requests.get(mars_weather_url)
# Create BeautifulSoup object; parse with 'lxml'
soup_level2 = BeautifulSoup(response.text, 'lxml')


<!DOCTYPE html>
<html data-scribe-reduced-action-queue="true" lang="en">
<head>
<meta charset="utf-8"/>
<script nonce="5ZUzGhU2EfPFtZbehzfFFQ==">
        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
      </script>
<script id="bouncer_terminate_iframe" nonce="5ZUzGhU2EfPFtZbehzfFFQ==">
    if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
<script id="swift_action_queue" nonce="5ZUzGhU2EfPFtZbehzfFFQ==">
    !function(){function e(e){if(e||(e=window.event),!e)return!1;if(e.timestamp=(new Date).getTime(),!e.target&&e.srcElement&&(e.target=e.srcElement),document.documentElement.getAttribute("data-scribe-reduced-action-queue"))for(var t=e.target;t&&t!=document.body;){if("A"==t.tagName)return;t=t.parentNode}return i("all",o(e)),a(e)?(document.addEventListener||(e=

In [135]:


def get_tweets(username):
    tweets = []
    URL = "https://twitter.com/"+username
    soup = BeautifulSoup(urllib.request.urlopen(URL), 'lxml')

    for li in soup.find_all("li", {"data-item-type": "tweet"}):
        text_p = li.find("p", class_="tweet-text")
        if text_p is not None:
            tweets.append(text_p.get_text())
    return tweets

get_tweets("marswxreport")

['Happy (UTC) New Year Earthlings.  It’s the Mars equivalent of Feb 2 (Ls 316.4) at 7:58 pm local (Gale Crater) time.\n\nIf we used the Julian calendar on Mars (and we dont) ~Jan 1 (Ls=180) was 2 Earth months ago on November 1, 2018.\n\nKeep up Earthlings!pic.twitter.com/oUVZJfWhLd',
 'Sol 2271 (2018-12-26), high -7C/19F, low -70C/-93F, pressure at 8.31 hPa, daylight 06:41-18:54',
 'Sol 2270 (2018-12-25), high -5C/23F, low -75C/-102F, pressure at 8.35 hPa, daylight 06:41-18:53',
 'Sol 2269 (2018-12-24), high -4C/24F, low -75C/-102F, pressure at 8.29 hPa, daylight 06:41-18:53',
 'BREAKING…Signal has just been received at mission control, New Horizons has successfully started its flyby program of stored commands and the exploration of Ultima Thule 4 billion miles away! THIS IS IT FOLKS, FLYBY HAS BEGUN! Go New Horizons!! \n#NASA #Science #SPACE#ULTIMAFLYBYpic.twitter.com/48VuCLHz5i',
 'Sol 2268 (2018-12-23), high -10C/14F, low -74C/-101F, pressure at 8.36 hPa, daylight 06:40-18:53',
 'So